# Rerankers

In this notebook, we will show how to use RedisVL to rerank search results (documents or chunks or records) based on an input query.

RedisVL for Java currently supports reranking through:

- A re-ranker that uses pre-trained [Cross-Encoders](https://sbert.net/examples/applications/cross-encoder/README.html) which can use models from [Hugging Face cross encoder models](https://huggingface.co/cross-encoder) or Hugging Face models that implement a cross encoder function ([example: BAAI/bge-reranker-base](https://huggingface.co/BAAI/bge-reranker-base)).

**Note:** CohereReranker and VoyageAIReranker from the Python version are not yet implemented in the Java version.

Before running this notebook, be sure to:
1. Have RedisVL JAR in your classpath
2. Have a running Redis Stack instance with RediSearch > 2.4 active

For example, you can run Redis Stack locally with Docker:

```bash
docker run -d -p 6379:6379 -p 8001:8001 redis/redis-stack:latest
```

This will run Redis on port 6379 and RedisInsight at http://localhost:8001.

In [1]:
// Load Maven dependencies
%maven com.microsoft.onnxruntime:onnxruntime:1.20.0
%maven com.google.code.gson:gson:2.11.0
%maven org.slf4j:slf4j-nop:2.0.16
%maven com.squareup.okhttp3:okhttp:4.12.0
%maven ai.djl.huggingface:tokenizers:0.30.0

// Note: RedisVL JAR must be in classpath (loaded automatically by Docker container)

In [2]:
// Import RedisVL reranking classes
import com.redis.vl.utils.rerank.*;

// Import Java standard libraries
import java.util.*;

## Simple Reranking

Reranking provides a relevance boost to search results generated by traditional (lexical) or semantic search strategies.

As a simple demonstration, take the passages and user query below:

In [3]:
String query = "What is the capital of the United States?";

List<String> docs = Arrays.asList(
    "Carson City is the capital city of the American state of Nevada. At the 2010 United States Census, Carson City had a population of 55,274.",
    "The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean that are a political division controlled by the United States. Its capital is Saipan.",
    "Charlotte Amalie is the capital and largest city of the United States Virgin Islands. It has about 20,000 people. The city is on the island of Saint Thomas.",
    "Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. The President of the USA and many major national government offices are in the territory. This makes it the political center of the United States of America.",
    "Capital punishment (the death penalty) has existed in the United States since before the United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states. The federal government (including the United States military) also uses capital punishment."
);

System.out.println("Query: " + query);
System.out.println("Number of documents: " + docs.size());

Query: What is the capital of the United States?
Number of documents: 5


The goal of reranking is to provide a more fine-grained quality improvement to initial search results. With RedisVL, this would likely be results coming back from a search operation like full text or vector.

## Using the Cross-Encoder Reranker

To use the cross-encoder reranker we initialize an instance of `HFCrossEncoderReranker` passing a suitable model.

In this notebook, we use `BAAI/bge-reranker-base` to match the Python notebook exactly. The Java implementation automatically detects the model architecture (BERT vs XLMRoberta) and handles tokenization accordingly.

**Note:** On first run, this will download the model (~280MB) and cache it in `~/.cache/redisvl4j/`. Subsequent runs will load from cache.

In [4]:
// Use BAAI/bge-reranker-base to match Python notebook
HFCrossEncoderReranker crossEncoderReranker = HFCrossEncoderReranker.builder()
    .model("BAAI/bge-reranker-base")
    .build();

System.out.println("HFCrossEncoderReranker initialized with model: " + crossEncoderReranker.getModel());

HFCrossEncoderReranker initialized with model: BAAI/bge-reranker-base


## Rerank documents with HFCrossEncoderReranker

With the obtained reranker instance we can rerank and truncate the list of documents based on relevance to the initial query.

**API Difference:** In Python, `rank()` returns a tuple `(results, scores)`. In Java, it returns a `RerankResult` object with `getDocuments()` and `getScores()` methods.

In [5]:
RerankResult result = crossEncoderReranker.rank(query, docs);

// Java API: result.getDocuments() and result.getScores()
// Python API: results, scores = reranker.rank(...)

List<?> results = result.getDocuments();
List<Double> scores = result.getScores();

for (int i = 0; i < results.size(); i++) {
    System.out.println(scores.get(i) + " -- " + results.get(i));
}

0.9999381303787231 -- Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. The President of the USA and many major national government offices are in the territory. This makes it the political center of the United States of America.
0.38023582100868225 -- Charlotte Amalie is the capital and largest city of the United States Virgin Islands. It has about 20,000 people. The city is on the island of Saint Thomas.
0.07461141049861908 -- Carson City is the capital city of the American state of Nevada. At the 2010 United States Census, Carson City had a population of 55,274.


Notice that the document about Washington, D.C. (the correct answer) is ranked first!

## Working with Semi-Structured Documents

Often times the initial result set includes other metadata and components.

**Important:** The Java implementation requires documents to have a `"content"` field containing the text to rank. The Python version with CohereReranker uses `rank_by` parameter and `"passage"` field, but the Java HFCrossEncoderReranker uses a fixed `"content"` field.

In [6]:
List<Map<String, Object>> structuredDocs = new ArrayList<>();

structuredDocs.add(Map.of(
    "source", "wiki",
    "content", "Carson City is the capital city of the American state of Nevada. At the 2010 United States Census, Carson City had a population of 55,274."
));

structuredDocs.add(Map.of(
    "source", "encyclopedia",
    "content", "The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean that are a political division controlled by the United States. Its capital is Saipan."
));

structuredDocs.add(Map.of(
    "source", "textbook",
    "content", "Charlotte Amalie is the capital and largest city of the United States Virgin Islands. It has about 20,000 people. The city is on the island of Saint Thomas."
));

structuredDocs.add(Map.of(
    "source", "textbook",
    "content", "Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. The President of the USA and many major national government offices are in the territory. This makes it the political center of the United States of America."
));

structuredDocs.add(Map.of(
    "source", "wiki",
    "content", "Capital punishment (the death penalty) has existed in the United States since before the United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states. The federal government (including the United States military) also uses capital punishment."
));

System.out.println("Created " + structuredDocs.size() + " structured documents");

Created 5 structured documents


In [7]:
// Rerank structured documents
// Note: Java HFCrossEncoderReranker does not support rank_by parameter like Python's CohereReranker
// It always uses the "content" field from the document maps

RerankResult structuredResult = crossEncoderReranker.rank(query, structuredDocs);

@SuppressWarnings("unchecked")
List<Map<String, Object>> rerankedResults = (List<Map<String, Object>>) structuredResult.getDocuments();
List<Double> structuredScores = structuredResult.getScores();

for (int i = 0; i < rerankedResults.size(); i++) {
    System.out.println(structuredScores.get(i) + " -- " + rerankedResults.get(i));
}

0.9999381303787231 -- {source=textbook, content=Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. The President of the USA and many major national government offices are in the territory. This makes it the political center of the United States of America.}
0.38023582100868225 -- {source=textbook, content=Charlotte Amalie is the capital and largest city of the United States Virgin Islands. It has about 20,000 people. The city is on the island of Saint Thomas.}
0.07461141049861908 -- {source=wiki, content=Carson City is the capital city of the American state of Nevada. At the 2010 United States Census, Carson City had a population of 55,274.}


Notice that all metadata fields (like `source`) are preserved in the reranked results.

## Cleanup

Close the reranker to free ONNX Runtime resources:

In [8]:
crossEncoderReranker.close();
System.out.println("Reranker closed successfully!");

Reranker closed successfully!


---

## Summary

This notebook demonstrates how to use HuggingFace cross-encoder models for reranking with RedisVL for Java. The implementation:

- ✅ Supports both BERT and XLMRoberta architectures  
- ✅ Automatically detects model architecture from config.json
- ✅ Handles both simple string documents and structured documents with metadata
- ✅ Provides feature parity with Python RedisVL reranking

## Python Version Features Not Yet Implemented

### CohereReranker
Uses the [Cohere /rerank API](https://docs.cohere.com/docs/rerank-2) for cloud-based reranking.

### VoyageAIReranker  
Uses the [VoyageAI /rerank API](https://docs.voyageai.com/docs/reranker) for cloud-based reranking.

These will be added in future releases.